In [7]:
import tensorflow_probability as tfp
import tensorflow as tf
import matplotlib.pyplot as plt
tfd = tfp.distributions

In [4]:
ndims = 2
step_std = 1.0
noise_std = 5.0
model = tfd.LinearGaussianStateSpaceModel(
  num_timesteps=100,
  transition_matrix=tf.linalg.LinearOperatorIdentity(ndims),
  transition_noise=tfd.MultivariateNormalDiag(
   scale_diag=step_std**2 * tf.ones([ndims])),
  observation_matrix=tf.linalg.LinearOperatorIdentity(ndims),
  observation_noise=tfd.MultivariateNormalDiag(
   scale_diag=noise_std**2 * tf.ones([ndims])),
  initial_state_prior=tfd.MultivariateNormalDiag(
   scale_diag=tf.ones([ndims])))

In [9]:
x = model.sample(500) # Sample from the prior on sequences of observations.
lp = model.log_prob(x) # Marginal likelihood of a (batch of) observations.

# Compute the filtered posterior on latent states given observations,
# and extract the mean and covariance for the current (final) timestep.
_, filtered_means, filtered_covs, _, _, _, _ = model.forward_filter(x)
current_location_posterior = tfd.MultivariateNormalTriL(
              loc=filtered_means[..., -1, :],
              scale_tril=tf.linalg.cholesky(filtered_covs[..., -1, :, :]))

# Run a smoothing recursion to extract posterior marginals for locations
# at previous timesteps.
posterior_means, posterior_covs = model.posterior_marginals(x)
initial_location_posterior = tfd.MultivariateNormalTriL(
              loc=posterior_means[..., 0, :],
              scale_tril=tf.linalg.cholesky(posterior_covs[..., 0, :, :]))

In [11]:
x

<tf.Tensor: shape=(500, 100, 2), dtype=float32, numpy=
array([[[-14.266793 ,  17.200935 ],
        [-47.576538 , -16.495077 ],
        [  4.095273 ,  45.26618  ],
        ...,
        [-31.606743 , -44.571392 ],
        [  8.8823185,  30.940338 ],
        [-31.081799 ,  38.64265  ]],

       [[-17.230404 , -14.700854 ],
        [ -3.5846717,  -5.9381866],
        [-13.28781  ,  10.133196 ],
        ...,
        [ 10.808541 , -49.71431  ],
        [ 33.261906 , -30.232077 ],
        [  6.963492 ,   2.7525597]],

       [[-12.644489 ,   2.675743 ],
        [ 22.114353 , -15.394979 ],
        [ -6.476296 ,  -1.0234746],
        ...,
        [ -8.397209 ,  12.330747 ],
        [-41.00025  , -14.831768 ],
        [-45.78073  , -45.20452  ]],

       ...,

       [[-13.383758 , -15.684601 ],
        [  1.9122952,   7.565019 ],
        [ 22.370209 , -31.473108 ],
        ...,
        [-22.579266 , -11.216031 ],
        [ -3.8216164,  29.186014 ],
        [ 66.99002  ,   6.829892 ]],

       [

In [31]:
def permute(vec):
    out_list = []
    if len(vec)==0:
        return []
    if len(vec)==1:
        return [vec]
    for i in range(0, len(vec)):
        popped_i = vec[:i] + vec[i+1:]
        # sub_list = permute(popped_i)
        for sub_list in permute(popped_i):
            out_list.append([vec[i]]+sub_list)
    return out_list
        
        
    

In [33]:
permute([1,2,3, 5])

[[1, 2, 3, 5],
 [1, 2, 5, 3],
 [1, 3, 2, 5],
 [1, 3, 5, 2],
 [1, 5, 2, 3],
 [1, 5, 3, 2],
 [2, 1, 3, 5],
 [2, 1, 5, 3],
 [2, 3, 1, 5],
 [2, 3, 5, 1],
 [2, 5, 1, 3],
 [2, 5, 3, 1],
 [3, 1, 2, 5],
 [3, 1, 5, 2],
 [3, 2, 1, 5],
 [3, 2, 5, 1],
 [3, 5, 1, 2],
 [3, 5, 2, 1],
 [5, 1, 2, 3],
 [5, 1, 3, 2],
 [5, 2, 1, 3],
 [5, 2, 3, 1],
 [5, 3, 1, 2],
 [5, 3, 2, 1]]